In [2]:
import os

import numpy as np
import trimesh
from tqdm import tqdm

import deep_sdf
import deep_sdf.workspace as ws
from custom_utils import *

# /cvlabdata2/home/remelli/DeepSDF/bin

In [3]:
# root = '/cvlabdata1/home/remelli/SVR/ShapeNetCore.v2/02958343/'
# save_dir = '/cvlabdata2/home/artem/Data/cars_refined'

# objects = list()
# for (dirpath, dirnames, filenames) in os.walk(root):
#     objects += [os.path.join(dirpath, file) for file in filenames if file == 'isosurf.obj']

In [4]:
# objects = ['/cvlabdata2/home/artem/DeepSDF/Expirements/OptimizationPaper/FairLoss/DeepSDFDrag/20/meshes/00000.ply,
#            '/cvlabdata2/home/artem/Expirements/SavedTransforms/DeepSDF-CFD8-car3-lift-min/meshes/00000.ply']
# save_dir = '/cvlabdata2/home/artem/Expirements/SavedTransforms/DeepSDF-CFD8-car3-lift-min/meshes_for_simulations'

In [5]:
def moveMesh(path_in, path_out, no_rotation=False):
    rotation_transform = np.array([[-1,  0,  0,  0], 
                                   [ 0,  1,  0,  0],
                                   [ 0,  0, -1,  0],
                                   [ 0,  0,  0,  1]], dtype=np.float)

    mesh = trimesh.load(path_in)
    mc = mesh.center_mass
    translation_matrix = np.array([[ 1,  0,  0,  -mc[0]], 
                                   [ 0,  1,  0,  -mc[1]],
                                   [ 0,  0,  1,  -mc[2]],
                                   [ 0,  0,  0,  1]], dtype=np.float)

    width = mesh.bounds[1][0] - mesh.bounds[0][0]
    sf = 1 / width
    scaling_matrix = np.array([[sf,  0,  0,  0], 
                               [ 0, sf,  0,  0],
                               [ 0,  0, sf,  0],
                               [ 0,  0,  0,  1]], dtype=np.float)

    if no_rotation:
        final_matrix = np.matmul(scaling_matrix, translation_matrix)
    else:
        final_matrix =  np.matmul(np.matmul(rotation_transform, translation_matrix), scaling_matrix)
    mesh.apply_transform(final_matrix)

    #np.save(os.path.join( os.path.join(save_dir, 'transforms'), car_name + '.npy'), final_matrix)
    #os.path.join( os.path.join(save_dir, 'cars'), car_name + '_input.stl')
    mesh.export(path_out)

In [14]:
moveMesh('/cvlabdata2/home/artem/DeepSDF/Expirements/OptimizationPaper/AfterMeeting/UmetamiDrag2/0535/meshes/00029.ply',
         '/cvlabdata2/home/artem/DeepSDF/Expirements/Simulations/TruckHole/inputs/umetami_29_input.stl', 
         no_rotation=True)

In [ ]:
reference_mesh_path = '/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/stl/0005.stl'
reference_mesh = trimesh.load(reference_mesh_path)

In [ ]:
reference_mesh.bounds

In [ ]:
import json
import os
import numpy as np

In [ ]:
files = list(filter(lambda x: x[-4:] == ".npz",  
                    os.listdir("data/SdfSamples/ShapeNetV2/02958343")))[:1500]
files = list(map(lambda x: x[:-4], files))
np.random.shuffle(files)

In [ ]:
train_set = {"ShapeNetV2" : {"02958343" : files[:-64]} }
test_set = {"ShapeNetV2" : {"02958343" : files[-64:]} }

In [ ]:
with open("examples/splits/sv2_cars_test.json", 'w') as file: 
    json.dump(test_set, file)
with open("examples/splits/sv2_cars_train.json", 'w') as file: 
    json.dump(train_set, file)

## Save DeepSDF learned shapes

In [9]:
experiment_directory = "/cvlabdata2/home/artem/DeepSDF/examples/cars_cleared/"
checkpoint = "latest"
decoder = load_model(experiment_directory, checkpoint)

latent_vectors = ws.load_latent_vectors(experiment_directory, checkpoint)
print(len(latent_vectors))

1205


In [12]:
save_dir = '/cvlabdata2/home/artem/Data/cars_remeshed_dsdf'

rotation_transform = np.array([[-1,  0,  0,  0], 
                               [ 0,  1,  0,  0],
                               [ 0,  0, -1,  0],
                               [ 0,  0,  0,  1]], dtype=np.float)

for idx, latent in enumerate(latent_vectors):
    with torch.no_grad():
        ply_mesh = create_mesh( decoder,
                                latent,
                                N=256,
                                max_batch=int(2 ** 18),
                                offset=None,
                                scale=None)
    mesh = trimesh.Trimesh(vertices = np.hstack(( ply_mesh['vertex']['x'][:, None], 
                                                  ply_mesh['vertex']['y'][:, None], 
                                                  ply_mesh['vertex']['z'][:, None])),
                           faces = ply_mesh['face']['vertex_indices']
                          )

    car_name = str(idx)
    mc = mesh.center_mass
    translation_matrix = np.array([[ 1,  0,  0,  -mc[0]], 
                                   [ 0,  1,  0,  -mc[1]],
                                   [ 0,  0,  1,  -mc[2]],
                                   [ 0,  0,  0,  1]], dtype=np.float)
    
    width = mesh.bounds[1][0] - mesh.bounds[0][0]
    sf = 1 / width
    scaling_matrix = np.array([[sf,  0,  0,  0], 
                               [ 0, sf,  0,  0],
                               [ 0,  0, sf,  0],
                               [ 0,  0,  0,  1]], dtype=np.float)

    final_matrix = np.matmul(np.matmul(rotation_transform, translation_matrix), scaling_matrix)
    mesh.apply_transform(final_matrix)

    np.save(os.path.join( os.path.join(save_dir, 'transforms'), car_name + '.npy'), final_matrix)
    mesh.export(os.path.join( os.path.join(save_dir, 'inputs'), car_name + '_input.stl') )